In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from sklearn.preprocessing import Imputer
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from mlxtend.classifier import StackingClassifier 
from sklearn.preprocessing import scale

In [4]:
train=pd.read_csv('train.csv')

In [5]:
test=pd.read_csv('test.csv')

In [6]:
# This prints out (rows, columns) in each dataframe
print('Train shape:', train.shape)
print('Test shape:', test.shape)

print('Columns:',train.columns)

Train shape: (595212, 59)
Test shape: (892816, 58)
Columns: Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'p

In [7]:
for i in train.columns:
    print(i,' - ',len(train[train[str(i)]==-1]),end='  ')
    if str(i)!='target' : print(len(test[test[str(i)]==-1]))

id  -  0  0
target  -  0  ps_ind_01  -  0  0
ps_ind_02_cat  -  216  307
ps_ind_03  -  0  0
ps_ind_04_cat  -  83  145
ps_ind_05_cat  -  5809  8710
ps_ind_06_bin  -  0  0
ps_ind_07_bin  -  0  0
ps_ind_08_bin  -  0  0
ps_ind_09_bin  -  0  0
ps_ind_10_bin  -  0  0
ps_ind_11_bin  -  0  0
ps_ind_12_bin  -  0  0
ps_ind_13_bin  -  0  0
ps_ind_14  -  0  0
ps_ind_15  -  0  0
ps_ind_16_bin  -  0  0
ps_ind_17_bin  -  0  0
ps_ind_18_bin  -  0  0
ps_reg_01  -  0  0
ps_reg_02  -  0  0
ps_reg_03  -  107772  161684
ps_car_01_cat  -  107  160
ps_car_02_cat  -  5  5
ps_car_03_cat  -  411231  616911
ps_car_04_cat  -  0  0
ps_car_05_cat  -  266551  400359
ps_car_06_cat  -  0  0
ps_car_07_cat  -  11489  17331
ps_car_08_cat  -  0  0
ps_car_09_cat  -  569  877
ps_car_10_cat  -  0  0
ps_car_11_cat  -  0  0
ps_car_11  -  5  1
ps_car_12  -  1  0
ps_car_13  -  0  0
ps_car_14  -  42620  63805
ps_car_15  -  0  0
ps_calc_01  -  0  0
ps_calc_02  -  0  0
ps_calc_03  -  0  0
ps_calc_04  -  0  0
ps_calc_05  -  0  0
ps_c

In [8]:
# We drop these variables as we don't want to train on them
# The other 57 columns are all numerical and can be trained on without preprocessing
target=train['target']
id_test=test['id']
train = train.drop(['target', 'id'], axis=1)
test = test.drop(['id'], axis=1)

In [9]:
train_cols=[str(i) for i in train.columns]
test_cols=[str(i) for i in test.columns]

In [10]:
imr=Imputer(missing_values=-1,strategy='median',axis=0)
imrX= imr.fit(train)
train = imrX.transform(train)

imrX= imr.fit(test)
test = imrX.transform(test)

In [11]:
train=pd.DataFrame(train,columns=train_cols)
test=pd.DataFrame(test,columns=test_cols)

In [12]:
print(train.shape)
print(test.shape)

(595212, 57)
(892816, 57)


In [13]:
# Take a random 20% of the dataset as validation data
#x_train, x_valid, y_train, y_valid = train_test_split(train, target, test_size=0.2, random_state=9440)
#print('Train samples: {} Validation samples: {}'.format(len(x_train), len(x_valid)))
#del train

In [14]:
# Convert our data into XGBoost format
d_train = xgb.DMatrix(train,target)
#d_valid = xgb.DMatrix(x_valid, y_valid)
d_test=xgb.DMatrix(test)
#del x_train,y_train,x_valid,y_valid

In [15]:
# Set xgboost parameters
params = {}
#params['objective'] = 'binary:logistic'
params['eta'] = 0.005
params['silent'] = True
params['max_depth'] = 8
params['subsample'] = 0.9
params['colsample_bytree'] = 0.9
params['nthread']=-1

In [16]:
# Define the gini metric - from https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703#5897
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# Create an XGBoost-compatible metric from Gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

# This is the data xgboost will test on after eachboosting round
#watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [13]:
import gc
gc.collect()

28

In [14]:
pca = PCA(n_components=20)

In [15]:
pca.fit(train)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [16]:
li=list(pca.explained_variance_ratio_)
li1=list(pca.components_)
li2=list(pca.explained_variance_ratio_)

In [17]:
su=0
for j,i in zip(li2,li):
    su+=i
    print(j,'  ',su)

0.909095348605    0.909095348605
0.0254527833334    0.934548131938
0.0110427811529    0.945590913091
0.00703742993468    0.952628343026
0.00629284445334    0.958921187479
0.00609652293308    0.965017710412
0.00531248333953    0.970330193752
0.00453955194758    0.974869745699
0.00361329020621    0.978483035905
0.00292604159998    0.981409077505
0.00239604959743    0.983805127103
0.00177729953233    0.985582426635
0.00166901797829    0.987251444613
0.00150066430582    0.988752108919
0.00148506747056    0.99023717639
0.0012969597192    0.991534136109
0.00120701479495    0.992741150904
0.00107437303817    0.993815523942
0.00104107986867    0.994856603811
0.000751197801774    0.995607801613


In [18]:
train_=pd.DataFrame()
test_=pd.DataFrame()
train_=pca.fit_transform(train)
test_=pca.fit_transform(test)

In [19]:
gc.collect()
print(train_.shape,test_.shape)

(595212, 20) (892816, 20)


In [33]:
cols=[reg,bina,ind,calc,car,cat]
print(train_.shape,test_.shape)

(595212, 20) (892816, 20)


In [28]:
print(len(train[[str(i) for i in reg]].values))
print(len(train[[str(i) for i in bina]].values))
print(len(train[[str(i) for i in ind]].values))
print(len(train[[str(i) for i in calc]].values))
print(len(train[[str(i) for i in car]].values))
print(len(train[[str(i) for i in cat]].values))
                    

595212
595212
595212
595212
595212
595212


In [1]:
from sklearn.cluster import MiniBatchKMeans

In [30]:
new_cols=['reg_cluster','bina_cluster','ind_cluster','calc_cluster','car_cluster','cat_cluster']

In [34]:
train_clusters=pd.DataFrame()
test_clusters=pd.DataFrame()
cnt=0
total_data=train+test
for i in new_cols:
    coords = np.vstack((total_data[[str(j) for j in cols[cnt]]].values))
    sample_ind = np.random.permutation(len(coords))
    kmeans = MiniBatchKMeans(n_clusters=5, batch_size=10000).fit(coords[sample_ind])
    train_clusters[str(i)]=kmeans.predict(train[[str(j) for j in cols[cnt]]].values)
    print(len(train_clusters[str(i)]),max(train_clusters[str(i)].values),min(train_clusters[str(i)].values))
    cnt+=1
cnt=0
for i in new_cols:
    coords = np.vstack((total_data[[str(j) for j in cols[cnt]]].values))
    sample_ind = np.random.permutation(len(coords))
    kmeans = MiniBatchKMeans(n_clusters=5, batch_size=10000).fit(coords[sample_ind])
    test_clusters[str(i)]=kmeans.predict(test[[str(j) for j in cols[cnt]]].values)
    print(len(test_clusters['reg_cluster']),max(test_clusters['reg_cluster'].values),min(test_clusters['reg_cluster'].values))
    cnt+=1    
print(train_clusters.shape,test_.shape)    

595212 9 0
595212 9 0
595212 9 0
595212 9 0
595212 9 0
595212 9 0
892816 9 0
892816 9 0
892816 9 0
892816 9 0
892816 9 0
892816 9 0
(595212, 6) (892816, 20)


In [38]:
print(train_.shape,test_.shape)
train_=pd.DataFrame(train_)
train_clusters=pd.DataFrame(train_clusters)
test_=pd.DataFrame(test_)
test_clusters=pd.DataFrame(test_clusters)

(595212, 20) (892816, 20)


In [40]:
train_data=pd.concat([train_,train_clusters],axis=1)
test_data=pd.concat([test_,test_clusters],axis=1)

In [44]:
del train,test,train_,train_clusters,test_,test_clusters

In [49]:
print(train_data.shape,test_data.shape)

(595212, 26) (892816, 26)


In [50]:
clf1=LogisticRegression(verbose=0,n_jobs=-1,C=0.00001)
clf2=svm.SVC(verbose=False,C=0.00001,probability=True)
clf3=RandomForestClassifier(verbose=0,n_jobs=-1)
clf4=KNeighborsClassifier(n_neighbors=2,n_jobs=-1)
clf5=GaussianNB()
clf6=DecisionTreeClassifier(criterion='entropy')
clf7=xgb.XGBClassifier(max_depth=6, n_estimators=800, learning_rate=0.01,nthread=-1)
clf8=GradientBoostingClassifier(n_estimators=400,learning_rate=.02,verbose=0)
clf9=AdaBoostClassifier(n_estimators=10,base_estimator=clf5
                            ,learning_rate=.05)
clf14=RidgeClassifier(alpha=0.00001)
clf10=StackingClassifier(classifiers=[clf3,clf14,clf5,clf6,clf8,clf7],
                         meta_classifier=clf1,use_probas=False)

clf11=StackingClassifier(classifiers=[clf6,clf7,clf14],
                        meta_classifier=clf1,use_probas=False)
clf12=StackingClassifier(classifiers=[clf10,clf11,clf5],
                          meta_classifier=clf1,use_probas=False)
clf13=StackingClassifier(classifiers=[clf5,clf11,clf3],meta_classifier=clf1)
models=[clf7,clf8,clf14,clf10,clf11,clf12,clf13]

name=['XGB','GradientBoost','Ridge Regression','Stacking'
      ,'Stacking2','2 Layer Stacking','Stacking 3']


In [ ]:
score=cross_val_score(clf7,train_data,target,cv=5,scoring='accuracy')
print(name[0],' : ',score.mean(),'  [',score.std(),']')

In [48]:
from sklearn.cross_validation import cross_val_score
cnt=0
for i in models:
    score=cross_val_score(i,train_data,target,cv=2,scoring='accuracy')
    print(name[cnt],' : ',score.mean(),'  [',score.std(),']')
    cnt+=1


XGB  :  0.963552482141   [ 0.0 ]


KeyboardInterrupt: 

In [19]:
# Train the model! We pass in a max of 10,000 rounds (with early stopping after 100)
# and the custom metric (maximize=True tells xgb that higher metric is better)
#mdl = xgb.train(params, d_train, 1300, maximize=True)
train=scale(train)
mdl1=LogisticRegression(C=.00001,n_jobs=-1,verbose=True,solver='sag')
mdl2=ridge_regression(C=.00001)

convergence after 17 epochs took 20 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   19.8s finished


LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=True, warm_start=False)

In [ ]:
mdl3=StackingClassifier()

In [22]:
# Predict on our test data
test=scale(test)
p_test = mdl1.predict_proba(test)

In [28]:
p_test=pd.DataFrame(p_test)
print(len(p_test[1]))

892816


In [29]:
# Create a submission file
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = p_test[1]
sub.to_csv('logistic_18_oct.csv', index=False)

In [18]:
import pickle

In [19]:
pickle.dump(mdl,open('logistic_18_oct.sav','wb'))

In [25]:
submit=pd.read_csv('logistic_18_oct.csv')

In [26]:
print((submit['id'][submit['target']<0]))

1903         3108
62373      103799
210793     351584
252643     421218
386513     644391
489806     816590
571960     953454
581210     969030
668561    1114605
711562    1186263
883140    1472018
Name: id, dtype: int64


In [27]:
submit['target'][submit['target']<0]=0.000000

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
submit.to_csv('xgb_17_oct.csv', index=False)

In [21]:
cat=[]
for i in train.columns:
    if 'cat' in i:
        cat.append(i)

In [22]:
bina=[]
for i in train.columns:
    if 'bin' in i:
        bina.append(i)

In [23]:
reg=[]
for i in train.columns:
    if 'reg' in i:
        reg.append(i)

In [24]:
ind=[]
for i in train.columns:
    if 'ind' in i and 'bin' not in i and 'cat' not in i:
        ind.append(i)

In [25]:
car=[]
for i in train.columns:
    if 'car' in i and 'bin' not in i and 'cat' not in i:
        car.append(i)

In [26]:
calc=[]
for i in train.columns:
    if 'calc' in i and 'bin' not in i and 'cat' not in i:
        calc.append(i)

In [27]:
colc=[reg,bina,ind,calc,car,cat]
print(train.columns,train.shape)
print(sum([len(i) for i in [reg,bina,ind,calc,car,cat]]))

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [4]:
print(corel)

[('ps_car_07_cat', -0.036395222587461001), ('ps_ind_06_bin', -0.034016869703860843), ('ps_car_02_cat', -0.031533596124388143), ('ps_ind_16_bin', -0.027778075788648455), ('ps_ind_15', -0.021506221687888169), ('ps_car_08_cat', -0.020341933485774941), ('ps_ind_09_bin', -0.0082368159855145673), ('ps_car_14', -0.0044741457023865817), ('ps_calc_19_bin', -0.001743831954662157), ('ps_car_11', -0.0012133568962230331), ('ps_calc_12', -0.0011325853981315429), ('ps_calc_20_bin', -0.001072228427533944), ('ps_calc_08', -0.0010058548384214974), ('ps_calc_15_bin', -0.00049042425066491422), ('ps_calc_13', -0.00044646453181066721), ('id', -0.0001884755338279302), ('ps_calc_17_bin', -0.00017012817366195574), ('ps_calc_07', -0.00010347690485265882), ('ps_calc_04', 3.2720455107159456e-05), ('ps_calc_06', 8.1822259782053064e-05), ('ps_calc_11', 0.0003714373948919784), ('ps_calc_18_bin', 0.00055243519962195513), ('ps_calc_16_bin', 0.00062382801150690024), ('ps_calc_09', 0.00071896758436575732), ('ps_calc_05'